In [1]:
!pip install dropconnect_tensorflow
!pip install tensorflow-addons

  Preparing metadata (setup.py) ... - \ done
  Created wheel for dropconnect_tensorflow: filename=dropconnect_tensorflow-0.1.1-py3-none-any.whl size=4640 sha256=32b34be7ebaac847cefd6334a24b6470d4196404dfef4eca7716a971cfbb4f59
  Stored in directory: /root/.cache/pip/wheels/7e/4a/e5/266cd645dbc3573352598ba045a07b58db04fc73b8cfab99ae
Successfully built dropconnect_tensorflow
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires typeguard<5,>=4.1.2, but you have typeguard 2.13.3 which is incompatible.


In [2]:
#Modelito con regularización y batch normalization, y  data augmentation
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from keras.datasets import mnist
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from PIL import Image
import cv2
import numpy as np
from tensorflow.keras.layers import Dense, Input
from dropconnect_tensorflow import DropConnectDense
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Lambda, Reshape
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()    

2024-02-24 01:06:34.773853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 01:06:34.773967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 01:06:34.910729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# One-hot encode labels
num_classes = 10
batch_size = 100
epochs = 15

# Reshape the data to add the channel dimension
x_train_split = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test_split = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

y_train_split = to_categorical(y_train, num_classes)
y_test_split = to_categorical(y_test, num_classes)

In [4]:
# Create an instance of ImageDataGenerator with the custom preprocessing function
datagen = ImageDataGenerator(
    rotation_range=10,       # Rotating -10°,10°
    width_shift_range=0.08,  # Shifting images to the width direction by 0.08
    height_shift_range=0.08, # Shifting images to the height direction by 0.08
    shear_range=0.03,        # 
    zoom_range=[0.8, 1.2],   # Scaling images by various factors between 0.8 and 1.2,
    zca_whitening=False,
    horizontal_flip=False, 
    vertical_flip=False,
    fill_mode = 'nearest'
)

datagen.fit(x_train_split)

# Input layer
inputs = Input(shape=(28, 28, 1))

# First Block
x = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='valid')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='valid')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.35)(x)
x = Conv2D(512, (3, 3), activation='relu', padding='valid')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Conv2D(1024, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)

# Second Block
x = Conv2D(2000, (3, 3), activation='relu', padding='valid')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.35)(x)

In [5]:
# Prepare to divide the feature maps into 10 subnets
shape = K.int_shape(x)
num_feature_maps = shape[-1]  # This would be 2000 based on your architecture
subnet_feature_maps = num_feature_maps // 10  # This assumes an even split

# Create subnetworks
subnet_outputs = []
for i in range(10):
    # Extract the subset of feature maps for this subnet
    subnet_input = Lambda(lambda z: z[:, :, :, i * subnet_feature_maps:(i + 1) * subnet_feature_maps])(x)
    
    # Reshape the feature maps to a 1D vector
    subnet_input = Reshape((shape[1] * shape[2] * subnet_feature_maps,))(subnet_input)
    
    # Subnetwork layers
    fc = Dense(512, activation='relu')(subnet_input)
    fc = BatchNormalization()(fc)
    fc = Dropout(0.5)(fc)
    fc = DropConnectDense(512, activation='relu', prob=0.5)(fc)
    subnet_output = Dense(10, activation='softmax')(fc)
    subnet_outputs.append(subnet_output)

stacked_subnet_outputs = tf.stack(subnet_outputs, axis=-1)

# Create the full model
full_model = Model(inputs=inputs, outputs=subnet_outputs)

# Summary of the model
full_model.summary()
#combined_output = Lambda(lambda outputs: K.mean(K.stack(outputs, axis=0), axis=0))(subnet_outputs)

# Define AdamW optimizer from TensorFlow addons
adamw_optimizer = tfa.optimizers.AdamW(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-8,
    weight_decay=0.0   # Set the weight decay rate here
)

# Compile the model

full_model.compile(loss='categorical_crossentropy', optimizer=adamw_optimizer, metrics=['accuracy'])

#earlystopper = EarlyStopping(monitor='val_loss', min_delta =0, patience=20, verbose=1, mode='min',restore_best_weights=True) 
# Callbacks
def scheduler(epoch, lr):
    if epoch < 15: # For the first 10 epochs, the learning rate is not changed
        return(lr)
    elif 15 < epoch < 30: # After ten is decreases exponentially
        return(lr*math.exp(-0.1))
    else:
        return(lr*math.exp(-0.2)) # And then decreases even more, still exponentially

lr_scheduler = LearningRateScheduler(scheduler)

checkpoint = ModelCheckpoint('mnist_best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

full_model.fit(
    datagen.flow(
    x_train_split, y_train_split, 
    batch_size=batch_size),
    steps_per_epoch=len(x_train_split) / batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test_split,  y_test_split),
    callbacks=[checkpoint,lr_scheduler]
)


# Evaluate the model
score = full_model.evaluate(x_test_split, y_test_split, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
full_model.save("modelinho.h5")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 26, 26, 64)           640       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 26, 26, 64)           256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 dropout (Dropout)           (None, 26, 26, 64)           0         ['batch_normalization[0][0

2024-02-24 01:06:55.243217: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


600/600 [==============================] - 77s 105ms/step - loss: 2.5650 - dense_1_loss: 0.2563 - dense_3_loss: 0.2563 - dense_5_loss: 0.2588 - dense_7_loss: 0.2552 - dense_9_loss: 0.2550 - dense_11_loss: 0.2559 - dense_13_loss: 0.2561 - dense_15_loss: 0.2559 - dense_17_loss: 0.2583 - dense_19_loss: 0.2573 - dense_1_accuracy: 0.9189 - dense_3_accuracy: 0.9185 - dense_5_accuracy: 0.9171 - dense_7_accuracy: 0.9184 - dense_9_accuracy: 0.9195 - dense_11_accuracy: 0.9183 - dense_13_accuracy: 0.9185 - dense_15_accuracy: 0.9195 - dense_17_accuracy: 0.9179 - dense_19_accuracy: 0.9184 - val_loss: 0.4572 - val_dense_1_loss: 0.0467 - val_dense_3_loss: 0.0459 - val_dense_5_loss: 0.0441 - val_dense_7_loss: 0.0464 - val_dense_9_loss: 0.0459 - val_dense_11_loss: 0.0456 - val_dense_13_loss: 0.0477 - val_dense_15_loss: 0.0452 - val_dense_17_loss: 0.0438 - val_dense_19_loss: 0.0458 - val_dense_1_accuracy: 0.9863 - val_dense_3_accuracy: 0.9871 - val_dense_5_accuracy: 0.9880 - val_dense_7_accuracy: 0.9871

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from tensorflow.keras.models import load_model
import numpy as np
from dropconnect_tensorflow import DropConnectDense

def manual_majority_vote(predictions):
    # Assuming predictions shape is (num_subnets, num_samples, num_classes)
    # We first transpose to get shape (num_samples, num_subnets, num_classes)
    predictions = np.transpose(predictions, (1, 0, 2))
    
    # Now we get the class with the highest probability from each subnet
    votes = np.argmax(predictions, axis=-1)  # Shape becomes (num_samples, num_subnets)
    
    # Apply majority voting across subnets
    final_predictions = np.array([np.bincount(votes[i]).argmax() for i in range(votes.shape[0])])
    return final_predictions

# Load the model with custom DropConnectDense layer
model_for_inference = load_model("modelinho.h5", custom_objects={"DropConnectDense": DropConnectDense})

# Making predictions with the model
# Ensure x_test_split is preprocessed as the model expects
predictions = model_for_inference.predict(x_test_split)  # Output shape should be (num_samples, num_subnets, num_classes)

# Check if predictions are a list of arrays and stack them if necessary
if isinstance(predictions, list):
    predictions = np.stack(predictions, axis=0)

# Apply majority voting to these predictions
final_predictions = manual_majority_vote(predictions)

# Assuming y_test_split is one-hot encoded, convert it to class indices for comparison
y_test_indices = np.argmax(y_test_split, axis=1)

# Calculate accuracy
accuracy = np.mean(final_predictions == y_test_indices)
print(f"Test Accuracy after Majority Voting: {accuracy * 100:.2f}%")


313/313 [==============================] - 4s 11ms/step
Test Accuracy after Majority Voting: 99.51%
